[Back to the starting page](start.ipynb)

In [1]:
from scripts import *
import inspect
from sklearn import linear_model

In [4]:
# Load data
x,y,elev                   = load_etopo(corrected_from_ice=True)
x,y,crustal_thickness      = load_crustal_thickness()
x,y,strain_rate            = load_strain_rate()
x,y,dist_closest_hs        = load_hotspots()
x,y,age                    = load_seafloor_ages()
x,y,lithospheric_thickness = load_lithospheric_thickness()
x,y,age_lithos             = load_age_lithos()

# correction for water load
elev = np.where(elev<0,elev-elev*1030/3400,elev)

min_dist_hs           = 1000e3 # m
threshold_strain_rate = 1e-16  # 1/s
ref_crustal_thickness = 35     # km

ETOPO 1 m min/max -9500.0 6065.0
Crustal thickness (GEMMA 2 arc-min) km min/max 0.009429668076336384 110.61520385742188
Strain rate (Kreemer et al, 2014) s-1 min/max 0.0 9.91755405912317e-13
Distances closest hot spot (Morgan and Morgan, 2007) m min/max 0.0 3931478.0
Seafloor age (Muller et al, 2008) Myrs min/max 0.0 278.9200134277344
Lithospheric thickness (SteinBerger 2016) km min/max 14.225520133972168 263.978759765625
Age lithosphere (Poupinet_Shapiro_2008) Ma min/max -1207.0679931640625 4128.31689453125


In [5]:
# Plot continental datasets
#
# plt.show/plt.pcolormesh is slow
#

%matplotlib widget

# Arguments description
print(inspect.getdoc(plot_global_maps_continents))

# Select data
# In case you want to select as a function of lon and lat, you must define xlon, ylat = np.meshgrid(x,y)
#
data                  = np.ma.masked_array(elev,
                                           ( (~age.mask) | 
                                             (crustal_thickness<ref_crustal_thickness-5) | 
                                             (crustal_thickness>ref_crustal_thickness+5) |
                                             (strain_rate>threshold_strain_rate)  | 
                                             (dist_closest_hs < min_dist_hs)))

# Plot
plot_global_maps_continents(x                     = x,
                            y                     = y,
                            data                  = data,
                            typedat               = 'elevation', # to set the display (xlim,ticks...)
                            plotcircles_hs        = True,
                            orientationbar        = 'vertical')

x,y : 1-D arrays
data : 2-D arrays
typedat = "distance_1st_hs", "seafloorage", "strainrate",
          "elevation", "crustal_thickness", "lithos_thickness",
          "age_lithos"
orientationbar = 'horizontal', 'vertical'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

masked_array(data=[712.4794311523438, 712.4794311523438,
                   712.4794311523438, ..., -108.74118041992188,
                   -108.74118041992188, -109.438232421875],
             mask=[False, False, False, ..., False, False, False],
       fill_value=1e+20,
            dtype=float32)

In [6]:
# Elevation distribution for stable continents with crustal thickness 35 +/- 5 km far from hotspots

%matplotlib widget

data = np.ma.masked_array(elev,
                               ( (~age.mask) | 
                                 (crustal_thickness<ref_crustal_thickness-5) | 
                                 (crustal_thickness>ref_crustal_thickness+5) |
                                 (strain_rate>threshold_strain_rate)  | 
                                 (dist_closest_hs < min_dist_hs)))

plot_histo(data[~data.mask].reshape(-1),
           'Elevation',
           xlabel='Elevation (m)',
           unit='m',
           GaussianModel=False,
           sigmamodel=175,
           meanmodel=175,
           text="right",
           xlim=[-1000,2500])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
# Elevation distribution for stable continents with crustal thickness +/- 5 km far from hotspots 
# excluding Antarctic

%matplotlib widget

# Excluding Antarctic removes the second mode
xlon, ylat = np.meshgrid(x,y)
data = np.ma.masked_array(elev,
                               ( (~age.mask) | 
                                 (crustal_thickness<ref_crustal_thickness-5) | 
                                 (crustal_thickness>ref_crustal_thickness+5) |
                                 (strain_rate>threshold_strain_rate)  | 
                                 (dist_closest_hs < min_dist_hs)      |
                                 (ylat<-60) ) )

plot_histo(data[~data.mask].reshape(-1),
           'Elevation',
           xlabel='Elevation (m)',
           unit='m',
           GaussianModel=True,
           sigmamodel=175,
           meanmodel=175,
           text="right",
           xlim=[-1000,2500])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
# Crustal thickness distribution for stable continents far from hotspots
%matplotlib widget

data = np.ma.masked_array(crustal_thickness,
                               ( (~age.mask) | 
                                 (strain_rate>threshold_strain_rate)  | 
                                 (dist_closest_hs < min_dist_hs)))

plot_histo(data[~data.mask].reshape(-1),
           'Crustal thickness',
           xlabel='Thickness (km)',
           unit='km',
           GaussianModel=True,
           sigmamodel=5,
           meanmodel=35,
           text="right",
           xlim=[5,75])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# Correlation Elevation versus crustal thickness for stable continents far from hotspots

mask          = ((age.mask) & (strain_rate<1e-16) & (dist_closest_hs > 1e6))
mask_default  = (age.mask)
count_default = np.count_nonzero(mask_default)
count         = np.count_nonzero(mask)
print('Stable continents far from hotspots represent {} % of the continental domain '.format(count*100/count_default))

y        = elev[mask]
x        = crustal_thickness[mask]

%matplotlib widget
title = 'elev continents vs crustal thickness'
plot_correlation(x,y,title,nbins = 20,ylabel='Elevation (m)',xlabel='Crustal thickness (km)',xlim=[15,65],ylim=[-1500,4000],ticks=[10,5,2000,250],
                 text="left",plottext=False)

Stable continents far from hotspots represent 68.8669376510444 % of the continental domain 


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
regr = linear_model.LinearRegression()
regr.fit(x.reshape(-1, 1), y)
xtest = np.linspace(15,65,100)
ypred = regr.predict(xtest.reshape(-1, 1))
plt.plot(xtest,ypred,label='Linear fit')
plt.legend()
plt.show()

In [13]:
# Correlation Elevation versus continental lithospheric thickness 
# for stable continents with crustal thickness 35 +/- 5 km far from hotspots

mask          = ((age.mask) & (crustal_thickness>30) 
                            & (crustal_thickness<40) 
                            & (strain_rate<1e-16)
                            & (dist_closest_hs > 1e6))
mask_default  = (age.mask)
count_default = np.count_nonzero(mask_default)
count         = np.count_nonzero(mask)
print('Stable continents with crustal thickness 35 +/- 5 km far from hotspots represent {} % of the continental domain '.format(count*100/count_default))

y        = elev[mask]
x        = lithospheric_thickness[mask]

%matplotlib widget
title = 'elev continents vs lithospheric thickness'
plot_correlation(x,y,title,
                 nbins = 20,
                 ylabel='Elevation (m)',
                 xlabel='Lithospheric thickness (km)',
                 xlim=[0,250],ylim=[-1000,2500],ticks=[50,25,2000,250],
                 text="left",plottext=False)

Stable continents with crustal thickness 35 +/- 5 km far from hotspots represent 30.238145289755483 % of the continental domain 


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
# Correlation Elevation versus continental lithospheric age 
# for stable continents with crustal thickness 35 +/- 5 km far from hotspots

mask          = ((age.mask) & (crustal_thickness>30) 
                            & (crustal_thickness<40) 
                            & (strain_rate<1e-16)
                            & (dist_closest_hs > 1e6))
mask_default  = (age.mask)
count_default = np.count_nonzero(mask_default)
count         = np.count_nonzero(mask)
print('Stable continents with crustal thickness 35 +/- 5 km far from hotspots represent {} % of the continental domain '.format(count*100/count_default))

y        = elev[mask]
x        = age_lithos[mask]

%matplotlib widget
title = 'elev continents vs lithospheric thickness'
plot_correlation(x,y,title,
                 nbins = 20,
                 ylabel='Elevation (m)',
                 xlabel='Lithospheric age (Ma)',
                 xlim=[0,4500],ylim=[-1000,2500],ticks=[500,250,2000,250],
                 text="left",plottext=False)

Stable continents with crustal thickness 35 +/- 5 km far from hotspots represent 30.238145289755483 % of the continental domain 


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …